In [28]:
import pmx, pmx.forcefield
import MDAnalysis as mda

In [29]:
itp_reorder_fname = "lipid14_popc_top/POPC_non-tail-atoms_q-sc_0.80_sig-sc_0.89_plamit-atoms-renamed.itp"
script_mapping_dir = "../../../scripts/atomic_partial_charges_compar_plot_l14_slipids_C36/"
itp_reference_fname = script_mapping_dir+"charmm_POPC.itp"

In [30]:
# ITP to be reordered
top_reorder = pmx.forcefield.ITPFile(fname=itp_reorder_fname)
# converts atom indices (as read from ITP file) 
# into corresponding Atom instances
# in all angles, dihedrals, bonds, vsites...
top_reorder.id2atoms()

# ITP of the reference (has correct ordering)
top_reference = pmx.forcefield.ITPFile(fname=itp_reference_fname)

In [31]:
# Directly taken from the script that compares partial charges of various POPC models

def fetch_atom(mol, at_name):
    """
    return the first occurence of an atom with a given name.
    return the 1st atom if searched atom is not found.
    """
    try:
        found = False
        for otherat in mol.atoms:
            if otherat.name == at_name:
                found = True
                #print "Found atom {atname}.".format(atname=at_name)
                return otherat
    except:
        print "Something went wrong during atom searching (function fetch_atom). \
        \nIs mol a pmx.forcefield.ITPfile object?"
    finally:
        if not found:
            print "Atom {atname} not found! -- will substitute it with atom no.1.".format(atname=at_name)
            # so that there's no missing space in the sequence.
            return mol.atoms[0]



Beware!!
---
Lipid14 convention of naming atoms is different from any other model. It contains different atoms with same names -- the difference is made by assigning them a different residue name: [PC, palmitoyl, oleoyl].

mapping_ab is hence ambiguous if the chains palmitoyl--oleoyl are not differenciated!

Here I use a special differenciated version of atom naming convention that append letter "p" to every atom in palmitoyl chaing beginning with glycerol C1 carbon (carbon the palmitoyl is attached to).

In [32]:
# Directly taken from the script that compares partial charges of various POPC models
# with only minor modifications

# translating A->B, so
# B is a reference being a dictionary in order  FFname - Mapping_name
# whereas A is being translated so a dictionary in opposite order is more practical
# A: Mapping_name -> FFname
# mappingPOPClipid14.txt -- works only for lipid14
mapping_xa = {}
with open(script_mapping_dir+"mappingPOPClipid14_palmit.txt","r") as f:
    for line in f.readlines():
        if not line.startswith("#"):
            items = line.split()
            mapping_xa[items[0]] = items[1]

# mappingFILE.txt -- works for slipids & charmm
mapping_bx = {}
with open(script_mapping_dir+"mappingFILE.txt","r") as f:
    for line in f.readlines():
        if not line.startswith("#"):
            items = line.split()
            mapping_bx[items[1]] = items[0]

# create dictionaries for transaltion from a->b
mapping_ab = {}
for key in mapping_bx.keys():
    mapping_ab[mapping_xa[mapping_bx[key]]] = key

mapping_ba = {}
for key in mapping_bx.keys():
    mapping_ba[key] = mapping_xa[mapping_bx[key]]


Simple solution: Converting conventions of atom naming using mapping files from Lipid14 -> Charmm36 convention

In [33]:
# a simple solution to the problem using mapping_ab can be used 
# only after the differentiation of atom names 
# between palmitoyl/oleoyl chains in LIPID14 nomenclature
top_reorder.atoms.sort(key=lambda a: fetch_atom(top_reference, mapping_ab[a.name]).id)
for i,a in enumerate(top_reorder.atoms):
    a.id = i+1

In [34]:
top_reorder.write("ecc-popc_C36-reordered.itp")

More complicated solution: using the backward mapping of atom names charmm -> lipid14

In [35]:
for a in top_reference.atoms:
    fetch_atom(top_reorder, mapping_ba[a.name]).id = a.id

Sorting the atoms of ECC-POPC according to the ordering of Charmm36 POPC

In [36]:
top_reorder.atoms.sort(key=lambda a: a.id)

In [37]:
top_reorder.write("ecc-popc_C36-reordered_ba.itp")

GOOD! both topologies are the same!